In [ ]:
%pip install geoai-py

In [ ]:
!apt-get update
!apt-get install gdal-bin -y

In [ ]:
import os
import geoai

In [ ]:
train_aerial_url = "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/las_vegas_train_naip.tif"
train_LiDAR_url = "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/las_vegas_train_hag.tif"
train_building_url = "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/las_vegas_buildings_train.geojson"
test_aerial_url = "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/las_vegas_test_naip.tif"
test_LiDAR_url = "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/las_vegas_test_hag.tif"

In [ ]:
train_aerial_path = geoai.download_file(train_aerial_url)
train_LiDAR_path = geoai.download_file(train_LiDAR_url)
train_building_path = geoai.download_file(train_building_url)
test_aerial_path = geoai.download_file(test_aerial_url)
test_LiDAR_path = geoai.download_file(test_LiDAR_url)

In [ ]:
from geoai.utils import view_image, view_raster, dict_to_image
view_image(train_aerial_path)

In [ ]:
os.environ["TITILER_ENDPOINT"] = "https://titiler.xyz"

In [ ]:
geoai.view_vector_interactive(train_building_path, tiles=train_aerial_url)

In [ ]:
geoai.view_vector_interactive(train_building_path, tiles=train_LiDAR_url)

In [ ]:
train_raster_path = "las_vegas_train_naip_hag.tif"
geoai.stack_bands(
    input_files=[train_aerial_path, train_LiDAR_path],
    output_file=train_raster_path,
    resolution=None,  # Automatically inferred from first image
    overwrite=True,
    dtype="Byte",  # or "UInt16", "Float32"
)

In [ ]:
test_raster_path = "las_vegas_test_naip_hag.tif"
geoai.stack_bands(
    input_files=[test_aerial_path, test_LiDAR_path],
    output_file=test_raster_path,
    resolution=None,  # Automatically inferred from first image
    overwrite=True,
    dtype="Byte",  # or "UInt16", "Float32"
)

In [ ]:
out_folder = "buildings"
tiles = geoai.export_geotiff_tiles(
    in_raster=train_raster_path,
    out_folder=out_folder,
    in_class_data=train_building_path,
    tile_size=512,
    stride=256,
    buffer_radius=0,
)

In [ ]:
view_image(train_raster_path)

In [ ]:
# Train U-Net model
geoai.train_segmentation_model(
    images_dir=f"{out_folder}/images",
    labels_dir=f"{out_folder}/labels",
    output_dir=f"{out_folder}/unet_models",
    architecture="unet",
    encoder_name="resnet34",
    encoder_weights="imagenet",
    num_channels=5,
    num_classes=2,  # background and building
    batch_size=8,
    num_epochs=50,
    learning_rate=0.001,
    val_split=0.2,
    verbose=True,
)

In [ ]:
geoai.plot_performance_metrics(
    history_path=f"{out_folder}/unet_models/training_history.pth",
    figsize=(15, 5),
    verbose=True,
)

**Performance metrics include**

1.   IoU (Intersection over union) or the Jaccard Index which measures the overlap between predicted region *A* and ground truth region *B* relative to their union, and
2.   Dice Score which measures
the overlap between *A* and *B*, but gives more weight to the intersection.


In [ ]:
# Define paths
masks_path = "building_masks.tif"
model_path = f"{out_folder}/unet_models/best_model.pth"

In [ ]:
# Run semantic segmentation inference
geoai.semantic_segmentation(
    input_path=test_raster_path,
    output_path=masks_path,
    model_path=model_path,
    architecture="unet",
    encoder_name="resnet34",
    num_channels=5,
    num_classes=2,
    window_size=512,
    overlap=256,
    batch_size=8,
)

In [ ]:
output_vector_path = "building_masks.geojson"
gdf = geoai.orthogonalize(masks_path, output_vector_path, epsilon=2)

In [ ]:
gdf_props = geoai.add_geometric_properties(gdf, area_unit="m2", length_unit="m")
print(f"Number of buildings: {len(gdf_props)}")

In [ ]:
geoai.view_raster(masks_path, nodata=0, basemap=test_aerial_url, backend="ipyleaflet")

In [ ]:
geoai.view_vector_interactive(gdf_props, column="area_m2", tiles=test_aerial_url)

In [ ]:
gdf_filtered = gdf_props[(gdf_props["area_m2"] > 50)]
print(f"Number of buildings: {len(gdf_filtered)}")
geoai.view_vector_interactive(gdf_filtered, column="area_m2", tiles=test_aerial_url)

In [ ]:
geoai.create_split_map(
    left_layer=gdf_filtered,
    right_layer=test_aerial_path,
    left_args={"style": {"color": "red", "fillOpacity": 0.2}},
    basemap=test_aerial_path,
)

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()

Support for third party widgets will remain active for the duration of the session. To disable support: